In [2]:
from datasets import load_dataset
import torch
from tokenizers import Tokenizer

num_titles = 10000
val_frac = 0.1
seed = 1337
ds = load_dataset("julien040/hacker-news-posts", split="train", cache_dir="./data").shuffle(seed=seed)
titles = [row["title"].strip() for row in ds.take(num_titles)]
n = int(num_titles * (1 - val_frac))
train_titles, val_titles= titles[:n], titles[n:]




c:\Users\teeds\miniconda3\envs\llm_train\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print('Train TITLES')
print(train_titles[0])
print('VAL TITles')
print(val_titles[0])

Train TITLES
Doom on Ubuntu Phone
VAL TITles
New Twists in the Road to Quantum Supremacy


In [4]:
from tokenizers import Tokenizer, decoders, models, pre_tokenizers, trainers    


def train_tokenizer(titles: list[str], vocab_size: int, unk_token: str = "<unk>", pad_token: str = "<pad>", eos_token: str = "<eos>") -> Tokenizer:
    tokenizer = Tokenizer(models.BPE(unk_token=unk_token))
    tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
    tokenizer.decoder = decoders.ByteLevel()
    trainer = trainers.BpeTrainer(
        vocab_size=vocab_size,
        special_tokens=[pad_token, eos_token, unk_token]
    )
    tokenizer.train_from_iterator(titles, trainer)
    return tokenizer




def get_batch(split_ids: torch.Tensor, ptr: int, block_size: int, batch_size: int, device: torch.device):
    span = block_size * batch_size + 1
    if ptr + span >= len(split_ids):
        ptr = 0
    batch = split_ids[ptr: ptr + span]
    x = batch[:-1].view(batch_size, block_size).to(device)
    y = batch[1:].view(batch_size, block_size).to(device)
    return x, y, ptr + block_size * batch_size


In [5]:
vocab_size = 16000

In [ ]:
train_tokenizer(train_titles+val_titles, vocab_size, eos_token="<eos>")

In [6]:
from model.tokenizer.BPETokenizer import BPETokenizer
tok = BPETokenizer(train_tokenizer(train_titles+val_titles, vocab_size, eos_token="<eos>"))

In [8]:
eos_token="<eos>"
train_text = eos_token.join(train_titles) + eos_token
val_text = eos_token.join(val_titles) + eos_token
train_ids = torch.tensor(tok.encode(train_text), dtype=torch.long)
val_ids = torch.tensor(tok.encode(val_text), dtype=torch.long)

In [12]:
len(train_titles) , len(val_titles)

(9000, 1000)

In [13]:
train_titles[0]

'Doom on Ubuntu Phone'

In [10]:
train_ids.shape, val_ids.shape

(torch.Size([107337]), torch.Size([11963]))

In [14]:
train_ids[:10]

tensor([ 6130,   274,  1866,  1723,     1,  5115,  5482,   246,  1228, 10936])